In [2]:
!pip install  PyPDF2 scikit-learn

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 10.9 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cac

In [3]:
import PyPDF2


def extract_text(pdf_file_path):
    # Initialize PDF reader
    pdf_reader = PyPDF2.PdfReader(pdf_file_path)

    # Extract text from PDF
    text = ""
    for page in pdf_reader.pages:
        text += page.extract_text()

    return text



# Path to  PDF file
pdf_path = "/content/drive/MyDrive/dataset/SADA.pdf"

# Extract text with formatting cues from PDF
text = extract_text(pdf_path)


/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [14]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS


def extract_keywords(text, num_keywords=10):
    # Initialize the TF-IDF Vectorizer
    tfidf_vectorizer = TfidfVectorizer(stop_words='english')

    # Fit and transform the text
    tfidf_matrix = tfidf_vectorizer.fit_transform([text])

    # Get the feature names (keywords)
    feature_names = tfidf_vectorizer.get_feature_names_out()

    # Get the scores of the features
    tfidf_scores = tfidf_matrix.toarray().flatten()

    # Create a DataFrame to display the keywords and their scores
    tfidf_df = pd.DataFrame({
        'keyword': feature_names,
        'score': tfidf_scores
    })

    # Sort the DataFrame by score in descending order
    tfidf_df = tfidf_df.sort_values(by='score', ascending=False)

    # Return the top N keywords
    keywords_df = tfidf_df.head(num_keywords)
    return keywords_df, keywords_df["keyword"].unique().tolist()

# Extract and print the top 10 keywords
keywords_df, predicted_keywords = extract_keywords(text, num_keywords=10)
print(keywords_df)

     keyword     score
393     loan  0.386014
20        12  0.373147
127     3601  0.321678
259  charges  0.205874
511   rupees  0.205874
287   credit  0.205874
231     bank  0.193007
249     card  0.167273
347     hdfc  0.154406
449  payment  0.141538


In [13]:
 predicted_keywords

['loan',
 '12',
 '3601',
 'charges',
 'rupees',
 'credit',
 'bank',
 'card',
 'hdfc',
 'payment']

In [15]:
import ast

predicted =str(predicted_keywords)

predicted = ast.literal_eval(predicted)

actual = ['charges', 'rupees', 'credit', 'bank', "hdfc","payment"]

#You need to pass input file object here
input_excel_file_name = pdf_path

In [22]:
import csv
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

def export_evalution_metrics_as_csv(input_excel_file_name, output_location, actual, predicted):

    actual = sorted(actual, key=len)
    predicted = sorted(predicted, key=len)

    tepm = input_excel_file_name

    input_excel_file_name = input_excel_file_name.split("/")[-1]
    input_excel_file_name = input_excel_file_name.split(".")[0]
    # Binarize the predicted and actual key terms
    all_terms = list(set(predicted + actual))
    y_true = [1 if term in actual else 0 for term in all_terms]
    y_pred = [1 if term in predicted else 0 for term in all_terms]

    # Calculate precision, recall, f1 score, and accuracy
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    accuracy = accuracy_score(y_true, y_pred)

    # Print the scores
    print(f"Precision: {precision:.2f}")
    print(f"Recall: {recall:.2f}")
    print(f"F1 Score: {f1:.2f}")
    print(f"Accuracy: {accuracy:.2f}")

    output_location += f"/{input_excel_file_name}_keytrems_evalution_metrics.csv"

    # Write the scores to a CSV file
    with open(output_location, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(["Extracted file name", temp])
        writer.writerow(["Actual keyterms :"] + actual)
        writer.writerow(["Predicted keyterms :"] + predicted)
        writer.writerow(["", ""])
        writer.writerow(["Metric", "Value"])
        writer.writerow(["Precision", precision])
        writer.writerow(["Recall", recall])
        writer.writerow(["F1 Score", f1])
        writer.writerow(["Accuracy", accuracy])

    print(f"Metrics written to {output_location}")



In [23]:
output_location = r"/content/drive/MyDrive/results"

export_evalution_metrics_as_csv(input_excel_file_name, output_location, actual, predicted)

Precision: 0.60
Recall: 1.00
F1 Score: 0.75
Accuracy: 0.60
Metrics written to /content/drive/MyDrive/results/SADA_keytrems_evalution_metrics.csv


In [24]:
#!pip install xlsxwriter

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 159.9/159.9 kB 1.3 MB/s eta 0:00:00


In [27]:
import pandas as pd
import xlsxwriter
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

def export_evalution_metrics_as_excel(input_excel_file_name, output_location, actual, predicted):
    actual = sorted(actual, key=len)
    predicted = sorted(predicted, key=len)

    temp = input_excel_file_name
    input_excel_file_name = input_excel_file_name.split("/")[-1]
    input_excel_file_name = input_excel_file_name.split(".")[0]
    # Binarize the predicted and actual key terms
    all_terms = list(set(predicted + actual))
    y_true = [1 if term in actual else 0 for term in all_terms]
    y_pred = [1 if term in predicted else 0 for term in all_terms]

    # Calculate precision, recall, f1 score, and accuracy
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    accuracy = accuracy_score(y_true, y_pred)

    # Print the scores
    print(f"Precision: {precision:.2f}")
    print(f"Recall: {recall:.2f}")
    print(f"F1 Score: {f1:.2f}")
    print(f"Accuracy: {accuracy:.2f}")

    output_file_path = f"{output_location}/{input_excel_file_name}_keyterms_evaluation_metrics.xlsx"

    # Create a workbook and add a worksheet
    workbook = xlsxwriter.Workbook(output_file_path)
    worksheet = workbook.add_worksheet()

    # Add a bold format to use for the headers
    bold = workbook.add_format({'bold': True})

    # Write the extracted file name
    worksheet.write('A1', 'Extracted file name', bold)
    worksheet.write('B1', temp)

    # Write the actual key terms
    worksheet.write('A2', 'Actual keyterms :', bold)
    worksheet.write_row('B2', actual)

    # Write the predicted key terms
    worksheet.write('A3', 'Predicted keyterms :', bold)
    worksheet.write_row('B3', predicted)

    # Leave a blank row
    worksheet.write('A4', '', bold)
    worksheet.write('B4', '')

    # Write the metrics headers
    worksheet.write('A5', 'Metric', bold)
    worksheet.write('B5', 'Value', bold)

    # Write the metrics
    metrics = [('Precision', precision), ('Recall', recall), ('F1 Score', f1), ('Accuracy', accuracy)]
    row = 5
    for metric, value in metrics:
        worksheet.write(row, 0, metric)
        worksheet.write(row, 1, value)
        row += 1

    workbook.close()

    print(f"Metrics written to {output_file_path}")


output_location =  r"/content/drive/MyDrive/results"

export_evalution_metrics_as_excel(input_excel_file_name, output_location, actual, predicted)

Precision: 0.60
Recall: 1.00
F1 Score: 0.75
Accuracy: 0.60
Metrics written to /content/drive/MyDrive/results/SADA_keyterms_evaluation_metrics.xlsx


In [ ]:
!pip install keybert PyPDF2 transformers

In [ ]:
import PyPDF2
from keybert import KeyBERT
import re

def extract_text(pdf_file_path):
    # Initialize PDF reader
    pdf_reader = PyPDF2.PdfReader(pdf_file_path)

    # Extract text from PDF
    text = ""
    for page in pdf_reader.pages:
        text += page.extract_text()

    return text



# Path to  PDF file
pdf_path = "/content/drive/MyDrive/dataset/SADA.pdf"

# Extract text with formatting cues from PDF
text = extract_text(pdf_path)

# Initialize KeyBERT model
kw_model = KeyBERT()

# Extract keywords
keywords = kw_model.extract_keywords(text)


In [ ]:
keywords

[('hdfcbank', 0.4359),
 ('loan', 0.4093),
 ('repayment24', 0.3744),
 ('insta', 0.3471),
 ('annum21', 0.3324)]

In [ ]:
text

'15-Mar-2024\nMr./Ms. PAMMI VENU MADHAV\nNO 2-190 DUDIYA TANDA CHLKUR M\nCHILKUR\nNALGONDA 508206\n\xa0\nDear Customer, \nSubject: Insta loan on your HDFC Bank Credit Card Number: 457704XXXXXX6561\n\xa0\nWe are pleased to grant you Insta loan as per the details stated below. The loan amount would be\ndisbursed to you as per your request.\nLoan no.\xa0\xa0\xa0 \xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa00000000000103733415\nUnsecured Loan Type \xa0\xa0\xa0\xa0\xa0\xa0\xa0Insta Loan\nAmount\xa0\xa0\xa0 \xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0Rs.70000.00\nFixed Interest Rate \xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa01.76%\xa0p.m.\nTenure \xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa024 Months\nEMI \xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0

In [ ]:
from keybert import KeyBERT

# Define input text
text =text

# Initialize KeyBERT model
model = KeyBERT('distilbert-base-nli-mean-tokens')

# Extract keywords using KeyBERT
#keywords = model.extract_keywords(text, keyphrase_ngram_range=(1, 1), stop_words='english', use_maxsum=True, nr_candidates=20, top_n=10)
keywords = model.extract_keywords(text)
# Print extracted keywords
print("Extracted Keywords:")
for keyword, _ in keywords:
    print(keyword)


Extracted Keywords:
202520335
202538650
202535729
202460270
202462766


In [ ]:
from keybert import KeyBERT

# Define input text
text = text

# Initialize KeyBERT model with different BERT-based models

# Example 1: distilbert-base-nli-stsb-mean-tokens
model_distilbert_stsb = KeyBERT('distilbert-base-nli-stsb-mean-tokens')
keywords_distilbert_stsb = model_distilbert_stsb.extract_keywords(text)

# Example 2: bert-base-nli-mean-tokens
model_bert_nli = KeyBERT('bert-base-nli-mean-tokens')
keywords_bert_nli = model_bert_nli.extract_keywords(text)

# Example 3: roberta-base-nli-mean-tokens
model_roberta_nli = KeyBERT('roberta-base-nli-mean-tokens')
keywords_roberta_nli = model_roberta_nli.extract_keywords(text)

# Example 5: sentence-transformers/bert-base-nli-mean-tokens
model_sentence_transformers = KeyBERT('sentence-transformers/bert-base-nli-mean-tokens')
keywords_sentence_transformers = model_sentence_transformers.extract_keywords(text)

# Print extracted keywords for each model
print("Keywords using distilbert-base-nli-stsb-mean-tokens:", [keyword for keyword, _ in keywords_distilbert_stsb])
print("Keywords using bert-base-nli-mean-tokens:", [keyword for keyword, _ in keywords_bert_nli])
print("Keywords using roberta-base-nli-mean-tokens:", [keyword for keyword, _ in keywords_roberta_nli])
print("Keywords using sentence-transformers/bert-base-nli-mean-tokens:", [keyword for keyword, _ in keywords_sentence_transformers])


Keywords using distilbert-base-nli-stsb-mean-tokens: ['disbursed', 'repayment', 'loans', 'lending', 'payments']
Keywords using bert-base-nli-mean-tokens: ['bankhttps', 'banking', 'borrowermonthly', 'loans', 'borrowerhttps']
Keywords using roberta-base-nli-mean-tokens: ['202520335', 'hdfcbank', 'lending', '202526655', 'repayment24']
Keywords using sentence-transformers/bert-base-nli-mean-tokens: ['bankhttps', 'banking', 'borrowermonthly', 'loans', 'borrowerhttps']


In [ ]:
#!pip install rake_nltk nltk

In [ ]:
from rake_nltk import Rake
from keybert import KeyBERT
import PyPDF2
import re
import nltk
nltk.download('stopwords')
nltk.download('punkt')

# Define input text
text = text

# Preprocess text and remove noise
# Tokenize text and remove stop words

# Generate candidate keywords using RAKE
r = Rake()
r.extract_keywords_from_text(text)
keywords_rake = r.get_ranked_phrases()

# Initialize KeyBERT model
#model = KeyBERT('distilbert-base-nli-mean-tokens')
model = KeyBERT()

# Extract keywords using KeyBERT
keywords_keybert = model.extract_keywords(text, keyphrase_ngram_range=(1, 1), stop_words='english', use_maxsum=True, nr_candidates=20, top_n=10)

# Select top-ranked keywords as final extracted keywords
final_keywords = [keyword for keyword, _ in keywords_keybert]

# Print final extracted keywords
print("Extracted Keywords:")
for keyword in final_keywords:
    print(keyword)


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Extracted Keywords:
approved
rupees
inst
tanda
3417
borrowermonthly
hdfc
loans
ixrepayment
nagar


In [ ]:

model = KeyBERT('roberta-base-nli-mean-tokens')

#keywords_keybert = model.extract_keywords(text, keyphrase_ngram_range=(1, 1), stop_words='english', use_maxsum=True, nr_candidates=20, top_n=10)
keywords_keybert = model.extract_keywords(text,stop_words='english')
# Select top-ranked keywords as final extracted keywords
final_keywords = [keyword for keyword, _ in keywords_keybert]

# Print final extracted keywords
print("Extracted Keywords:")
for keyword in final_keywords:
    print(keyword)

Extracted Keywords:
202520335
hdfcbank
lending
202526655
repayment24


In [ ]:
model = KeyBERT('sentence-transformers/bert-base-nli-mean-tokens')

#keywords_keybert = model.extract_keywords(text, keyphrase_ngram_range=(1, 1), stop_words='english', use_maxsum=True, nr_candidates=20, top_n=10)
keywords_keybert = model.extract_keywords(text,stop_words='english')
# Select top-ranked keywords as final extracted keywords
final_keywords = [keyword for keyword, _ in keywords_keybert]

# Print final extracted keywords
print("Extracted Keywords:")
for keyword in final_keywords:
    print(keyword)

Extracted Keywords:
bankhttps
banking
borrowermonthly
loans
borrowerhttps


In [ ]:
model = KeyBERT('bert-base-nli-mean-tokens')

#keywords_keybert = model.extract_keywords(text, keyphrase_ngram_range=(1, 1), stop_words='english', use_maxsum=True, nr_candidates=20, top_n=10)
keywords_keybert = model.extract_keywords(text,stop_words='english')
# Select top-ranked keywords as final extracted keywords
final_keywords = [keyword for keyword, _ in keywords_keybert]

# Print final extracted keywords
print("Extracted Keywords:")
for keyword in final_keywords:
    print(keyword)

Extracted Keywords:
bankhttps
banking
borrowermonthly
loans
borrowerhttps


In [ ]:
keywords_rake

['0000000000103733415 unsecured loan type insta loan amount rs',
 'nodal grievance redressal officer designated specifically',
 'please contact phonebanking within 7 days',
 'net disbursed amount using irr approach',
 'immediate billing cycle post loan booking',
 '7ae1be57c87f /? path =/ personal',
 '7ae1be57c87f /? path =/ common',
 'bank holidays xvii privacy policies',
 'payment shall attract late payment charges',
 'chilkur nalgonda 508206 dear customer',
 'monthly hdfc bank credit card statement',
 'hdfc bank credit card get closed',
 'last statement date till date',
 'overall credit card billing statement',
 'emi billing dateoutstandi ng principal',
 'emi billing dateoutstandi ng principal',
 'provided )- vnet disbursed amount',
 '18 %)* processing fee rs',
 'immediate billing cycle date',
 'tenure 24 months emi rs',
 'hdfc bank credit card number',
 '00 fixed interest rate 1',
 '00 (* gst 18 %),',
 'balance principal outstanding plus gst',
 'lal bahadur shastri marg',
 '2024 mr 

In [ ]:
from rake_nltk import Rake
import PyPDF2
from keybert import KeyBERT
import re
import nltk
nltk.download('stopwords')
nltk.download('punkt')


def extract_text(pdf_file_path):
      # Initialize PDF reader
      pdf_reader = PyPDF2.PdfReader(pdf_file_path)

      # Extract text from PDF
      text = ""
      for page in pdf_reader.pages:
          text += page.extract_text()

      return text

def generate_candidate_keywords_using_rake(text):
    # Generate candidate keywords using RAKE
    r = Rake()
    r.extract_keywords_from_text(text)
    keywords_rake = r.get_ranked_phrases()
    return keywords_rake

def generate_contextual_keywords_using_llm(text, llm_model = None):
    model = KeyBERT()
    if llm_model != None:
       model = KeyBERT(llm_model)

    #keywords_keybert = model.extract_keywords(text, keyphrase_ngram_range=(1, 1), stop_words='english', use_maxsum=True, nr_candidates=20, top_n=10)
    keywords_keybert = model.extract_keywords(text,stop_words='english')
    return keywords_keybert

def get_top_ranked_keywords(keywords_rake, keywords_keybert):
    final_keywords = [keyword for keyword in keywords_rake if keyword in keywords_keybert]











